I've written a simple sentential logic evaluator using a binary tree implementation. 
Here is the Binary Tree class:

In [1]:
import operator as op
import pdb

class BinaryTree:

    def __init__(self, rootObj):
        self.key = rootObj
        self.parent = None
        self.leftChild = None
        self.rightChild = None

    def insertLeft(self, newNode):
        if self.leftChild == None:
            t = BinaryTree(newNode)
            self.leftChild = t
            t.parent = self
        else:
            t = BinaryTree(newNode)
            t.parent = self
            t.leftChild = self.leftChild
            self.leftChild.parent = t
            self.leftChild = t

    def insertRight(self, newNode):
        if self.rightChild == None:
            t = BinaryTree(newNode)
            self.rightChild = t
            t.parent = self
        else:
            t = BinaryTree(newNode)
            t.parent = self
            t.rightChild = self.rightChild
            self.rightChild.parent = t
            self.rightChild = t

    def getRightChild(self):
        return self.rightChild

    def getLeftChild(self):
        return self.leftChild

    def getParent(self):
        return self.parent

    def getRootVal(self):
        return self.key

    def setRootVal(self, obj):
        self.key = obj

    def setLeftChild(self, nodeObj):
        self.leftChild = nodeObj

    def setRightChild(self, nodeObj):
        self.rightChild = nodeObj

    def setParent(self, nodeObj):
        self.parent = nodeObj
        
    def isRightChild(self):
        return self.parent and self.parent.rightChild == self
    
    def isLeftChild(self):
        return self.parent and self.parent.leftChild == self
    def isRoot(self):
        return

Simple inorder traversal:

In [2]:
def inorder_traversal(tree):
    if tree:
        inorder_traversal(tree.getLeftChild())
        print(tree.getRootVal())
        inorder_traversal(tree.getRightChild())

Now I had to built a syntax tree, it builds a tree similar to one uses in evaluation of arithmetic expressions; the difference is in negation operator (it's unary) - creates only one new node:

In [3]:
implication = lambda x, y: op.or_(op.not_(x), y)

def build_parse_tree(exp):
    exp_list = exp.replace('(', ' ( ').replace(')', ' ) ').replace('~', ' ~ ').split()
    e_tree = BinaryTree('')
    current_tree = e_tree
    cnt = 0
    for token in exp_list:
        if token == '(':
                current_tree.insertLeft('')
                current_tree = current_tree.getLeftChild()
        elif token in ['||','&&', '->']:
            if current_tree.getRootVal() == '~':
                current_tree.getParent().setRootVal(token)
                current_tree.insertRight('')
                current_tree = current_tree.getRightChild()
            else:
                current_tree.setRootVal(token)
                current_tree.insertRight('')
                current_tree = current_tree.getRightChild()
        elif token in ['1', '0']:
            current_tree.setRootVal(bool(int(token)))
            current_tree = current_tree.getParent()
            if current_tree.getRootVal() == '~':
                current_tree = current_tree.getParent()
        elif token == '~':
            current_tree.setRootVal('~')
            current_tree.insertLeft('')
            current_tree = current_tree.getLeftChild()
        elif token == ')':
            current_tree = current_tree.getParent()
        else:
            raise ValueError
        cnt += 1
    return e_tree

Time to eval function:

In [4]:
def evaluate_parse_tree(tree):
    opers = {'||': op.or_, '&&': op.and_, '~': op.not_, '->': implication} 
    leftT = tree.getLeftChild()
    rightT = tree.getRightChild()
    if leftT and not rightT:
        fn = opers[tree.getRootVal()]
        return fn(evaluate_parse_tree(leftT))
    elif leftT and rightT:
        fn = opers[tree.getRootVal()]
        return fn(evaluate_parse_tree(leftT), evaluate_parse_tree(rightT))
    else:
        #pdb.set_trace()
        return tree.getRootVal()

Eval picks an operator from a dict and recursively call itself (from both branches - binary operand, from left only - unary negation)   
Examples:

In [5]:
tree1 = build_parse_tree('(1 ->(0 -> 1))')

This is a tree:

In [6]:
inorder_traversal(tree1)

True
->
False
->
True


And finally evaluation gives true:

In [7]:
evaluate_parse_tree(tree1)

True

Another one: 

In [8]:
tree2 = build_parse_tree('~((1 -> (0 -> 0)) -> ((1 -> 0) -> (1 -> 0)))')

In [9]:
inorder_traversal(tree2)

True
->
False
->
False
->
True
->
False
->
True
->
False
~


In [10]:
evaluate_parse_tree(tree2)

False

One more tree:

In [11]:
tree3 = build_parse_tree('((~1 -> ~0) -> (0 -> 1))')

In [12]:
inorder_traversal(tree3)

True
~
->
False
~
->
False
->
True


In [13]:
evaluate_parse_tree(tree3)

True

Finall remarks:    
Because of negation building parse tree is bit messy.    
I don't see any other way to implement logical variables into this - because of parsing process; whole expression is a string and only bool(int('1')) or zero makes sense; bool('string') gives true always except empty string.    
Usability of this; obviously one may check if a formula is a thesis of calculus, with some effort it can be used to check if a given formula is a tautology (I'm thinking about this in some future posts).